In [1]:
import numpy as np
import time
import json
import os
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Check CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


Using device: cuda
GPU: NVIDIA GeForce RTX 3060


In [2]:
path = "../../../../data/multiclass/processed/"

datasets = ["mnist_multi4_pca_4",
            "mnist_multi4_pca_8",
        ]

# Multiple seeds for timing variation
seeds = [42, 100, 20, 5, 99]

sample_sizes = [100, 250, 400]

In [3]:
class ShallowNN(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim=4):
        super(ShallowNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
        # Calculate total parameters
        self.total_params = sum(p.numel() for p in self.parameters())
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    
    def get_param_count(self):
        return self.total_params

def train_model(model, train_loader, criterion, optimizer, epochs, device):
    """Train the neural network."""
    model.train()
    for epoch in range(epochs):
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            # Forward pass
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def evaluate_model(model, X, y, device):
    """Evaluate the model and return predictions."""
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X).to(device)
        outputs = model(X_tensor)
        _, predictions = torch.max(outputs, 1)
        predictions = predictions.cpu().numpy()
    return predictions


def remap_labels(y):
    
    unique_labels = np.unique(y)
    if len(unique_labels) != 4:
        raise ValueError(f"Expected 4 unique labels, got {len(unique_labels)}: {unique_labels}")
    
    # Create mapping: first unique label -> 0, second -> 1
    label_map = {unique_labels[0]: 0, unique_labels[1]: 1, unique_labels[2]: 2, unique_labels[3]: 3}
    return np.array([label_map[label] for label in y])


# Hyperparameter search space
hidden_dims = [8, 16, 32]  # Will give ~50-100 params depending on input_dim
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [32, 64]
epochs_options = [50, 100]

In [4]:
# Initialize results storage
results_path = "../../../../results/classical_nn_4way_baseline_results.json"
if os.path.exists(results_path):
    with open(results_path, 'r') as f:
        all_results = json.load(f)
    print(f"Loaded existing results with {len(all_results['results'])} experiments")
else:
    all_results = {
        "experiment_info": {
            "model_type": "shallow_neural_network",
            "date": datetime.now().isoformat(),
            "architecture": "Single hidden layer with ReLU activation",
            "hyperparameter_tuning": "Grid search over hidden_dim, learning_rate, batch_size, epochs",
            "optimizer": "Adam",
            "device": str(device)
        },
        "results": []
    }
    print("Created new results file")

# Main experiment loop
for dataset in datasets:
    print(f"\n{'='*70}")
    print(f"Dataset: {dataset}")
    
    # Load data
    X_full = np.load(os.path.join(path, f"{dataset}/X_train.npy"))
    y_full = np.load(os.path.join(path, f"{dataset}/y_train.npy"))
    X_test = np.load(os.path.join(path, f"{dataset}/X_test.npy"))
    y_test = np.load(os.path.join(path, f"{dataset}/y_test.npy"))
    
    
    y_full = remap_labels(y_full)
    y_test = remap_labels(y_test)
   
    
    print(f"Available training samples: {X_full.shape[0]}")
    print(f"Input dimension: {X_full.shape[1]}")
    print(f"Unique labels in training: {np.unique(y_full)}")  # Should be [0, 1]
    print(f"Unique labels in test: {np.unique(y_test)}")      # Should be [0, 1]
    print(f"{'='*70}\n")
    
    input_dim = X_full.shape[1]
    
    # Test different sample sizes
    for n_samples in sample_sizes:
        if n_samples > X_full.shape[0]:
            n_samples = X_full.shape[0]
        
        print(f"{'─'*70}")
        if n_samples == X_full.shape[0]:
            print(f"Training with FULL dataset ({n_samples} samples)")
        else:
            print(f"Training with {n_samples} samples (subsampled)")
        print(f"{'─'*70}")
        
        for seed in seeds:
            
            experiment_exists = any(
                r["dataset"] == dataset and 
                r["n_train"] == n_samples and 
                r["seed"] == seed
                for r in all_results["results"]
            )
            
            if experiment_exists:
                print(f"  Seed {seed:3d}: SKIPPING (already exists)")
                continue
            
            # Subsample if needed
            if n_samples < X_full.shape[0]:
                indices = np.random.RandomState(seed).choice(
                    X_full.shape[0], n_samples, replace=False
                )
                X_train = X_full[indices]
                y_train = y_full[indices]
            else:
                X_train = X_full
                y_train = y_full
            
            # Split into train/validation (80/20)
            X_train_split, X_val, y_train_split, y_val = train_test_split(
                X_train, y_train, test_size=0.2, random_state=seed, stratify=y_train
            )
            
            # Set random seeds for reproducibility
            torch.manual_seed(seed)
            np.random.seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed(seed)
            
            # Hyperparameter tuning via grid search on validation set
            best_val_acc = 0
            best_params = {}
            
            for hidden_dim in hidden_dims:
                for lr in learning_rates:
                    for batch_size in batch_sizes:
                        for epochs in epochs_options:
                            # Create model
                            model = ShallowNN(input_dim, hidden_dim).to(device)
                            
                            # Skip if parameter count is too far from target (50-100)
                            param_count = model.get_param_count()
                            if param_count > 150:  # Allow some flexibility
                                continue
                            
                            # Create data loaders
                            train_dataset = TensorDataset(
                                torch.FloatTensor(X_train_split),
                                torch.LongTensor(y_train_split)
                            )
                            train_loader = DataLoader(
                                train_dataset,
                                batch_size=batch_size,
                                shuffle=True
                            )
                            
                            # Train
                            criterion = nn.CrossEntropyLoss()
                            optimizer = optim.Adam(model.parameters(), lr=lr)
                            train_model(model, train_loader, criterion, optimizer, epochs, device)
                            
                            # Validate
                            y_val_pred = evaluate_model(model, X_val, y_val, device)
                            val_acc = accuracy_score(y_val, y_val_pred)
                            
                            # Track best
                            if val_acc > best_val_acc:
                                best_val_acc = val_acc
                                best_params = {
                                    'hidden_dim': hidden_dim,
                                    'learning_rate': lr,
                                    'batch_size': batch_size,
                                    'epochs': epochs,
                                    'param_count': param_count
                                }
            
            # Train final model with best hyperparameters on full training set
            print(f"  Seed {seed:3d}: Best params - hidden={best_params['hidden_dim']}, "
                  f"lr={best_params['learning_rate']}, batch={best_params['batch_size']}, "
                  f"epochs={best_params['epochs']}, params={best_params['param_count']}")
            
            # Set seeds again for final training
            torch.manual_seed(seed)
            np.random.seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed(seed)
            
            final_model = ShallowNN(input_dim, best_params['hidden_dim']).to(device)
            
            # Create data loader for full training set
            train_dataset = TensorDataset(
                torch.FloatTensor(X_train),
                torch.LongTensor(y_train)
            )
            train_loader = DataLoader(
                train_dataset,
                batch_size=best_params['batch_size'],
                shuffle=True
            )
            
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
            
            # Training time
            start_time = time.time()
            train_model(final_model, train_loader, criterion, optimizer, 
                       best_params['epochs'], device)
            training_time = time.time() - start_time
            
            # Inference time
            start_time = time.time()
            y_pred = evaluate_model(final_model, X_test, y_test, device)
            inference_time = time.time() - start_time
            
            # Metrics
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='macro')
            
            result = {
                "dataset": dataset,
                "n_train": int(n_samples),
                "n_test": int(X_test.shape[0]),
                "n_features": int(X_train.shape[1]),
                "seed": int(seed),
                "best_hidden_dim": int(best_params['hidden_dim']),
                "best_learning_rate": float(best_params['learning_rate']),
                "best_batch_size": int(best_params['batch_size']),
                "best_epochs": int(best_params['epochs']),
                "param_count": int(best_params['param_count']),
                "accuracy": float(accuracy),
                "f1_score": float(f1),
                "training_time_seconds": float(training_time),
                "inference_time_seconds": float(inference_time),
                "timestamp": datetime.now().isoformat()
            }
            
            all_results["results"].append(result)
            
            print(f"            Test - Acc={accuracy:.4f}, F1={f1:.4f}, "
                  f"Train={training_time:.2f}s")

Created new results file

Dataset: mnist_multi4_pca_4
Available training samples: 400
Input dimension: 4
Unique labels in training: [0 1 2 3]
Unique labels in test: [0 1 2 3]

──────────────────────────────────────────────────────────────────────
Training with 100 samples (subsampled)
──────────────────────────────────────────────────────────────────────
  Seed  42: Best params - hidden=8, lr=0.01, batch=64, epochs=100, params=76
            Test - Acc=0.8800, F1=0.8790, Train=0.27s
  Seed 100: Best params - hidden=16, lr=0.01, batch=32, epochs=50, params=148
            Test - Acc=0.8800, F1=0.8813, Train=0.26s
  Seed  20: Best params - hidden=8, lr=0.001, batch=32, epochs=50, params=76
            Test - Acc=0.8100, F1=0.8073, Train=0.26s
  Seed   5: Best params - hidden=8, lr=0.01, batch=32, epochs=100, params=76
            Test - Acc=0.9100, F1=0.9085, Train=0.51s
  Seed  99: Best params - hidden=8, lr=0.01, batch=32, epochs=50, params=76
            Test - Acc=0.8800, F1=0.8802, 

In [6]:
# Save results
os.makedirs("../../results", exist_ok=True)
with open(results_path, 'w') as f:
    json.dump(all_results, indent=2, fp=f)

print(f"\n{'='*70}")
print(f"Results saved to {results_path}")
print(f"Total experiments: {len(all_results['results'])}")
print(f"{'='*70}")


Results saved to ../../../../results/classical_nn_4way_baseline_results.json
Total experiments: 30
